# Dynamic Tests

This jupyter notebook allows you to treat a video so that in its upper left corner appears the name of the label that the neural network thinks you are seeing at that moment.

In [31]:
# OS imports
import glob
import os,shutil
import re
# Keras imports
from keras.models import load_model
from keras.preprocessing import image as preprocessing
# Image processing imports
from PIL import Image, ImageFont, ImageDraw, ImageOps
import numpy as np
import cv2
# Other
import time

# My directories paths
commonPath = ""
videos = commonPath + "dataset/test/originals"
saveInDir = commonPath + "dataset/test/results"

## Video Map

In this cell we will map our video frames.

With a "for" i'm going to iterate and between X and Y frames i will assign a Z label.

In [32]:

mapVideoTest1=[]
numTotalFrames = 6500
for i in range(0, 770):
    mapVideoTest1.append(0)
for i in range(770, 1112):
    mapVideoTest1.append(1)
for i in range(1112, 1282):
    mapVideoTest1.append(-1)
for i in range(1282, 2315):
    mapVideoTest1.append(2)
for i in range(2315, 2571):
    mapVideoTest1.append(3)
for i in range(2571, 2763):
    mapVideoTest1.append(2)
for i in range(2763, 4484):
    mapVideoTest1.append(4)
for i in range(4484, 5668):
    mapVideoTest1.append(5)
for i in range(5668, 6348):
    mapVideoTest1.append(6)
for i in range(6348,numTotalFrames):
    mapVideoTest1.append(2)
    
mapVideoTest2=[]
numTotalFrames = 5512
for i in range(0, 385):
    mapVideoTest2.append(0)
for i in range(385, 1050):
    mapVideoTest2.append(1)
for i in range(1050, 1210):
    mapVideoTest2.append(-1)
for i in range(1210, 2010):
    mapVideoTest2.append(2)
for i in range(2010, 4200):
    mapVideoTest2.append(4)
for i in range(4200, 4900):
    mapVideoTest2.append(5)
for i in range(4900, 5350):
    mapVideoTest2.append(6)
for i in range(5350, numTotalFrames):
    mapVideoTest2.append(3)


mapVideoTest3=[]
numTotalFrames = 10000
for i in range(0, 1110):
    mapVideoTest3.append(0)
for i in range(1110,1741):
    mapVideoTest3.append(-1)
for i in range(1741,2187):
    mapVideoTest3.append(1)
for i in range(2187,2373):
    mapVideoTest3.append(-1)
for i in range(2373,3541):
    mapVideoTest3.append(2)
for i in range(3541,3589):
    mapVideoTest3.append(-1)
for i in range(3589,4317):
    mapVideoTest3.append(3)
for i in range(4317,4397):
    mapVideoTest3.append(-1)
for i in range(4397,7030):
    mapVideoTest3.append(4)
for i in range(7030,8050):
    mapVideoTest3.append(5)
for i in range(8050,8620):
    mapVideoTest3.append(6)
for i in range(8620,9000):
    mapVideoTest3.append(3)


mapVideoTest4=[]
numTotalFrames = 7790
for i in range(0, 1125):
    mapVideoTest4.append(0)
for i in range(1125,1155):
    mapVideoTest4.append(-1)
for i in range(1155,2212):
    mapVideoTest4.append(1)
for i in range(2212,2260):
    mapVideoTest4.append(-1)
for i in range(2260,3580):
    mapVideoTest4.append(2)
for i in range(3580,3628):
    mapVideoTest4.append(-1)
for i in range(3628,4019):
    mapVideoTest4.append(3)
for i in range(4019,4067):
    mapVideoTest4.append(-1)
for i in range(4067,6125):
    mapVideoTest4.append(4)
for i in range(6125,6300):
    mapVideoTest4.append(-1)
for i in range(6300,7025):
    mapVideoTest4.append(5)
for i in range(7025,7156):
    mapVideoTest4.append(-1)
for i in range(7156,7364):
    mapVideoTest4.append(3)
for i in range(7364,7400):
    mapVideoTest4.append(-1)
for i in range(7400,7790):
    mapVideoTest4.append(6)
    
mapVideoTest5=[]
numTotalFrames = 7790
for i in range(0, 256):
    mapVideoTest5.append(6)
for i in range(256, 846):
    mapVideoTest5.append(5)
for i in range(846,3020):
    mapVideoTest5.append(4)
for i in range(3020,3290):
    mapVideoTest5.append(3)
for i in range(3290,3626):
    mapVideoTest5.append(2)
for i in range(3626,3652):
    mapVideoTest5.append(-1)
for i in range(3652,3938):
    mapVideoTest5.append(3)
for i in range(3938,4838):
    mapVideoTest5.append(1)
for i in range(4838,5273):
    mapVideoTest5.append(0)

## New video creation

With all this funtions you can select a video and use your neural network to predict frame by frame what the neural network think is watching.

In [39]:
def testVideo(videoPath, realLabelsMap):
    # We split the path and select nly the name of the .mp4
    videoName = videoPath.split('/')[3]
    cap = cv2.VideoCapture(videoPath)
    
    # We read the dimensions of the video and we minimize the new video size (less MB).
    frameWidth = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frameHeight = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    newVideoFramesSize = (384, 216)
    
    # Temporal folder where we are going to sae the frames of the new video.
    try:
        os.stat('.temp')
    except:
        os.mkdir('.temp')
    
    totalTrues = 0
    frameNumber = 0
    while(cap.isOpened()):
        ret, frame = cap.read()
        if ret:
            #for every frame of the video...
            printAndRemove("Working on frame nº" + str(frameNumber))
            
            # Preprocessing of the image for the Neural Network.
            img = generateNeuralNetworkImage(frame, neuralNetworkImageSize)
            # Use Neural Network to predict the node.
            labelPredicted = translateNode(lookForBest(model.predict(img)))
            # We update the accuracy.
            if(translateNode(realLabelsMap[frameNumber]) == labelPredicted or realLabelsMap[frameNumber] == -1):
                totalTrues = totalTrues + 1
            # Write label in the new frame.
            newFrame = generateNewVideoFrame(
                frame, 
                newVideoFramesSize,
                labelPredicted, 
                translateNode(realLabelsMap[frameNumber]), 
                totalTrues/(frameNumber+1)
            )
            
            # Save new frame in temporal folder.
            cv2.imwrite(".temp/frame" + str(frameNumber) + ".jpg", cv2.cvtColor(newFrame, cv2.COLOR_BGR2RGB))
            frameNumber = frameNumber + 1
        else:
            break
    print("")
    
    # We create the new video with the new frames from the temporal folder (with the label).
    createNewVideoWithFrames(videoName, ".temp/*.jpg", newVideoFramesSize)
    # We move the video to the saveInDir folder.
    shutil.move(videoName, saveInDir)
    # Close all.
    cap.release()
    cv2.destroyAllWindows()
    # Remove temporal folder.
    shutil.rmtree('.temp/')
        
    # Print results.
    print("Video \"" + videoName + "\" ready")
    print("\t-True Positives: " + str(totalTrues))
    print("\t-Total Number Frames: " + str(frameNumber))
    print("\t-Accuracy: " + str(totalTrues/frameNumber))
    
# Print and remove the print
def printAndRemove(msg):
    print(msg, end="\r")
    time.sleep(0)
    
# Select the label most likely to be true
def lookForBest(prediction):
    best = 0
    categories = 6
    for i,probability in enumerate(prediction[0]):
        if probability > prediction[0][best]:
            best = i
    return best

# Use the neural network to predict what can be the image
def predictImg(img,realLabel,model):
    prediction = model.predict(img)
    return lookForBest(prediction)

# Translate the label number to his string
def translateNode(node):
    if node == -1:
        return "(Not mapped - Ignore)"
    if node == 0:
        return "Hall - Resident Rooms"
    if node == 1:
        return "Hall - Stairs"
    if node == 2:
        return "Building - Dinning room"
    if node == 3:
        return "Building - Rooms"
    if node == 4:
        return "Parking"
    if node == 5:
        return "Building - Library"
    if node == 6:
        return "Building - Gym"
    return "Unknown"
    
# The image must be refined before used in the neural network. It must have the right dimensions.
def generateNeuralNetworkImage(image, size):
    img = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    img = Image.fromarray(img).convert('RGB')
    img.thumbnail(neuralNetworkImageSize, Image.ANTIALIAS)
    img = np.expand_dims(img, axis=0)
    return img
    
# We create the new frame and we write the text we need on it.
def generateNewVideoFrame(frame, size, labelPredicted, labelReal, accuracy, ):
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    frame = Image.fromarray(frame).convert('RGB')
    #frame = frame.rotate(180)
    frame.thumbnail(size, Image.ANTIALIAS)
    frameDraw = ImageDraw.Draw(frame)
    frameDraw.rectangle( [(0, 0), (200, 50)], fill=(0,0,0) )
    frameDraw.text( (5, 5) , "Prediction:   " + labelPredicted,  font = ImageFont.truetype("arial.ttf", 12), fill=(255,255,255))
    frameDraw.text( (5, 20) , "Real Label: " + labelReal,  font = ImageFont.truetype("arial.ttf", 12), fill=(255,255,255))
    frameDraw.text( (5, 35) , "Accuracy: " + str(accuracy),  font = ImageFont.truetype("arial.ttf", 12), fill=(255,255,255))
    frame = np.array(frame)
    return frame

# We iterate in every frame from the temporal folder and write a new video.
def createNewVideoWithFrames(newVideoName, dirFrames, videoSize):
    video = cv2.VideoWriter(newVideoName, cv2.VideoWriter_fourcc(*'DIVX'), 30.0, videoSize)
    filenames = glob.glob(dirFrames)
    filenames.sort(key=recompile)

    for i,image in enumerate(filenames):
        printAndRemove("New Video - Working on frame nº" + str(i))
        video.write(cv2.imread(image))
    video.release()
    print("")

# Funtion to sort the images form the temporal folder.
def recompile(name):
    r= re.compile("(\d+).*").split(name)
    return [int(y) if y.isdigit() else y for y in r]

In [40]:
model = load_model(commonPath + 'Neural_Network.h5')
#neuralNetworkImageSize = (96, 54)
neuralNetworkImageSize = (128, 72)

# We execute the dinamic test code and create the new video.
testVideo(videos + "/test1.mp4", mapVideoTest1)
testVideo(videos + "/test2.mp4", mapVideoTest2)
testVideo(videos + "/test3.mp4", mapVideoTest3)
testVideo(videos + "/test4.mp4", mapVideoTest4)
testVideo(videos + "/test5.mp4", mapVideoTest5)

Working on frame nº6486
New Video - Working on frame nº6486
Video "test1.mp4" ready
	-True Positives: 5079
	-Total Number Frames: 6487
	-Accuracy: 0.7829505164174503
Working on frame nº5511
New Video - Working on frame nº5511
Video "test2.mp4" ready
	-True Positives: 5001
	-Total Number Frames: 5512
	-Accuracy: 0.9072931785195936
Working on frame nº8978
New Video - Working on frame nº8978
Video "test3.mp4" ready
	-True Positives: 5479
	-Total Number Frames: 8979
	-Accuracy: 0.6102015814678695
Working on frame nº7789
New Video - Working on frame nº7789
Video "test4.mp4" ready
	-True Positives: 7199
	-Total Number Frames: 7790
	-Accuracy: 0.9241335044929396
Working on frame nº5272
New Video - Working on frame nº5272
Video "test5.mp4" ready
	-True Positives: 4339
	-Total Number Frames: 5273
	-Accuracy: 0.822871230798407
